# 20 Build Municipio Hazard Features

Stage: `02_feature_engineering`
Discipline: hazard feature fusion and station-to-municipio aggregation.

Dependencies:
- `outputs/index_pipeline/01_ingest/flood_station_latest_features.csv`
- `outputs/index_pipeline/01_ingest/nws_alerts_enriched.csv`
- `outputs/index_pipeline/01_ingest/usgs_earthquake_events.csv`
- `outputs/index_pipeline/10_features/municipio_exposure_vulnerability_features.csv`

Output:
- `outputs/index_pipeline/20_features/municipio_hazard_features.csv`


In [ ]:
# Cell 1: Setup
import importlib.util
import subprocess
import sys
from pathlib import Path
import logging
import math
from datetime import datetime, timezone


def ensure_packages(packages):
    missing = [p for p in packages if importlib.util.find_spec(p) is None]
    if missing:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *missing])


ensure_packages(["pandas", "numpy"])

import numpy as np
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("index-pipeline-stage20")


def find_repo_root():
    p = Path.cwd().resolve()
    for c in [p, *p.parents]:
        if (c / "JupyterNotebooks").exists():
            return c
    return p


REPO_ROOT = find_repo_root()
BASE_OUT = REPO_ROOT / "JupyterNotebooks" / "outputs" / "index_pipeline"
INGEST_DIR = BASE_OUT / "01_ingest"
FEATURES10_DIR = BASE_OUT / "10_features"
OUTPUT_DIR = BASE_OUT / "20_features"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

try:
    from IPython.display import display
except ImportError:
    display = print


In [ ]:
# Cell 2: Load dependencies
station_file = INGEST_DIR / "flood_station_latest_features.csv"
alerts_file = INGEST_DIR / "nws_alerts_enriched.csv"
eq_file = INGEST_DIR / "usgs_earthquake_events.csv"
muni_file = FEATURES10_DIR / "municipio_exposure_vulnerability_features.csv"

for f in [station_file, alerts_file, eq_file, muni_file]:
    if not f.exists():
        raise FileNotFoundError(f"Missing dependency: {f}")

station_df = pd.read_csv(station_file)
alerts_df = pd.read_csv(alerts_file)
eq_df = pd.read_csv(eq_file)
muni_df = pd.read_csv(muni_file)

station_df["latest_time_utc"] = pd.to_datetime(station_df["latest_time_utc"], utc=True, errors="coerce")
eq_df["time_utc"] = pd.to_datetime(eq_df["time_utc"], utc=True, errors="coerce")

print(f"Stations: {len(station_df)} | Alerts: {len(alerts_df)} | Earthquakes: {len(eq_df)} | Municipios: {len(muni_df)}")


In [ ]:
# Cell 3: Build hazard features by municipio
LAMBDA_KM = float(25.0)  # distance decay scale
STATION_OVERRIDE_KM = float(12.0)


def haversine_km(lat1, lon1, lat2, lon2):
    r = 6371.0
    p1 = math.radians(lat1)
    p2 = math.radians(lat2)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(p1) * math.cos(p2) * math.sin(dlon / 2) ** 2
    return 2 * r * math.asin(math.sqrt(a))


def robust_to_0_100(series):
    x = pd.to_numeric(series, errors="coerce")
    lo = x.quantile(0.05)
    hi = x.quantile(0.95)
    if pd.isna(lo) or pd.isna(hi) or hi <= lo:
        return pd.Series(np.nan, index=x.index)
    return ((x - lo) / (hi - lo)).clip(0, 1) * 100

nws_global_alert = float(alerts_df["alert_score"].max()) if len(alerts_df) else 0.0
run_utc = datetime.now(timezone.utc)

rows = []
for _, m in muni_df.iterrows():
    mlat = pd.to_numeric(m.get("latitude"), errors="coerce")
    mlon = pd.to_numeric(m.get("longitude"), errors="coerce")
    if pd.isna(mlat) or pd.isna(mlon):
        continue

    sdf = station_df.copy()
    sdf["dist_km"] = sdf.apply(lambda r: haversine_km(mlat, mlon, float(r["lat"]), float(r["lon"])), axis=1)
    sdf["w"] = np.exp(-sdf["dist_km"] / LAMBDA_KM)

    weighted_flood = np.average(sdf["flood_hazard_final"], weights=sdf["w"]) if len(sdf) else np.nan

    nearby = sdf[sdf["dist_km"] <= STATION_OVERRIDE_KM]
    local_max = nearby["flood_hazard_final"].max() if len(nearby) else np.nan

    flood_hazard_muni = np.nanmax([weighted_flood, local_max, nws_global_alert])

    # Earthquake proxy: max event severity weighted by recency and depth
    eq_local = eq_df.copy()
    if len(eq_local):
        eq_local["dist_km"] = eq_local.apply(
            lambda r: haversine_km(mlat, mlon, float(r["latitude"]), float(r["longitude"])), axis=1
        )
        eq_local["age_h"] = (run_utc - eq_local["time_utc"]).dt.total_seconds() / 3600.0
        eq_local["depth_factor"] = 1.0 / (1.0 + pd.to_numeric(eq_local["depth_km"], errors="coerce").fillna(0) / 70.0)
        eq_local["recency_factor"] = np.exp(-eq_local["age_h"].clip(lower=0) / 168.0)
        eq_local["intensity_raw"] = (
            pd.to_numeric(eq_local["magnitude"], errors="coerce").fillna(0)
            / np.log1p(eq_local["dist_km"].clip(lower=1e-3))
            * eq_local["depth_factor"]
            * eq_local["recency_factor"]
        )
        eq_raw = eq_local["intensity_raw"].max()
    else:
        eq_raw = np.nan

    rows.append({
        "municipio": m["municipio"],
        "municipio_key": m["municipio_key"],
        "latitude": mlat,
        "longitude": mlon,
        "flood_hazard_weighted": weighted_flood,
        "flood_hazard_local_max": local_max,
        "nws_global_alert_score": nws_global_alert,
        "flood_hazard_muni": flood_hazard_muni,
        "earthquake_raw": eq_raw,
        "supporting_station_count": len(sdf),
        "nearby_station_count": int(len(nearby)),
    })

haz_df = pd.DataFrame(rows)
haz_df["earthquake_hazard_score"] = robust_to_0_100(haz_df["earthquake_raw"]).fillna(0)
haz_df["hazard_combined"] = haz_df[["flood_hazard_muni", "earthquake_hazard_score"]].max(axis=1)

# freshness proxy from latest noaa obs
if len(station_df):
    latest_obs = pd.to_datetime(station_df["latest_time_utc"], utc=True, errors="coerce").max()
else:
    latest_obs = pd.NaT
haz_df["noaa_latest_obs_utc"] = latest_obs

out_file = OUTPUT_DIR / "municipio_hazard_features.csv"
haz_df.to_csv(out_file, index=False)

print(f"Output: {out_file}")
display(haz_df.head(10))
